In [21]:
import ccxt 
import os
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

MONGO_URI = os.getenv("MONGO_URI")
FTX_API_KEY = os.getenv("FTX_API_KEY")
FTX_API_SECRET = os.getenv("FTX_API_SECRET")
SUBACCOUNT = 'qt-0003'

ftx = ccxt.ftx(
    {
        'apiKey': FTX_API_KEY,
        'secret': FTX_API_SECRET,
        'enableRateLimit': True,
        'headers': {
            'FTX-SUBACCOUNT': SUBACCOUNT,
        },
    }
)


In [22]:
data = ftx.fetch_my_trades()
trades = pd.DataFrame(data)

In [23]:
trades_cleaned = pd.json_normalize(trades['info'])
trades_cleaned = trades_cleaned.drop(['id', 'side', 'fee', 'price'], axis=1)
trades = pd.concat([trades.drop(['info'], axis=1), trades_cleaned], axis=1)
trades['price'] = trades['price'].astype(float)
trades['size'] = trades['size'].astype(float)
display(trades)
# trades_cleaned = trades[trades['market'].isin(tickers)]

# create index based on Market and side
trades_cleaned.set_index(['market', 'side'], inplace=True) 
trades_cleaned = trades_cleaned.drop(['id', 'fee', 'feeCurrency'], axis=1)

,timestamp,datetime,symbol,id,order,type,takerOrMaker,side,price,amount,...,baseCurrency,quoteCurrency,type,size,orderId,time,tradeId,feeRate,feeCurrency,liquidity
0,1666120047239,2022-10-18T19:07:27.239Z,BAL/USD:USD-221230,10441930390,191378908732,None,maker,buy,5.0650,0.010,...,None,None,order,0.010,191378908732,2022-10-18T19:07:27.239808+00:00,5172614620,-5e-6,USD,maker
1,1666120052639,2022-10-18T19:07:32.639Z,BAL/USD:USD-221230,10441931683,191378917781,None,maker,buy,5.0625,0.010,...,None,None,order,0.010,191378917781,2022-10-18T19:07:32.639209+00:00,5172615264,-5e-6,USD,maker
2,1666120075254,2022-10-18T19:07:55.254Z,BAL/USD:USD-221230,10441935610,191379001621,None,maker,buy,5.0625,0.010,...,None,None,order,0.010,191379001621,2022-10-18T19:07:55.254357+00:00,5172617203,-5e-6,USD,maker
3,1666120077348,2022-10-18T19:07:57.348Z,BAL/USD:USD-221230,10441935940,191379030672,None,maker,buy,5.0625,0.010,...,None,None,order,0.010,191379030672,2022-10-18T19:07:57.348227+00:00,5172617364,-5e-6,USD,maker
4,1666120084267,2022-10-18T19:08:04.267Z,BAL/USD:USD-221230,10441937069,191379057976,None,maker,buy,5.0625,1.000,...,None,None,order,1.000,191379057976,2022-10-18T19:08:04.267356+00:00,5172617927,-5e-6,USD,maker
5,1666120084754,2022-10-18T19:08:04.754Z,BAL/USD:USD-221230,10441937535,191379054529,None,maker,buy,5.0600,1.000,...,None,None,order,1.000,191379054529,2022-10-18T19:08:04.754814+00:00,5172618160,-5e-6,USD,maker
6,1666120099757,2022-10-18T19:08:19.757Z,BAL/USD:USD-221230,10441940665,191379129255,None,maker,buy,5.0575,1.000,...,None,None,order,1.000,191379129255,2022-10-18T19:08:19.757511+00:00,5172619715,-5e-6,USD,maker
7,1666120140877,2022-10-18T19:09:00.877Z,BAL/USD:USD-221230,10441948387,191379308999,None,maker,buy,5.0700,1.000,...,None,None,order,1.000,191379308999,2022-10-18T19:09:00.877685+00:00,5172623547,-5e-6,USD,maker
8,1666120283380,2022-10-18T19:11:23.380Z,BAL/USD:USD-221230,10441976469,191379922443,None,maker,buy,5.0650,1.000,...,None,None,order,1.000,191379922443,2022-10-18T19:11:23.380813+00:00,5172637482,-5e-6,USD,maker
9,1666120462798,2022-10-18T19:14:22.798Z,BAL/USD:USD-221230,10442005360,191380632515,None,maker,buy,5.0550,0.010,...,None,None,order,0.010,191380632515,2022-10-18T19:14:22.798840+00:00,5172651816,-5e-6,USD,maker


KeyError: "None of ['side'] are in the columns"